In [1]:
import sys
import os
from typing import Dict, Tuple
from src.utils import default_data_path, config_to_env, check_config
from src.task import LaMPTask
from src.models import (
    feed_prompt_to_lm,
    feed_prompts_to_lm,
    OpenAIModel,
    task_1_parse_response,
    DistilBERTModel,
    BERTSERINIModel,
    MiniLM,
)

In [2]:
config_to_env("OPENAI_API_KEY")

In [3]:
task_header = "LaMP_1"
store_folder = os.path.join("src", "data", task_header)

OpenAI_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=OpenAIModel(),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "OpenAI.txt"),
    callback=task_1_parse_response,
)
DistilBERT_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=DistilBERTModel(task_name="LaMP_1"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "DistilBERTModel.txt"),
    callback=task_1_parse_response,
)
BERTSERINI_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=BERTSERINIModel(task_name="LaMP_1"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "BERTSERINI.txt"),
    callback=task_1_parse_response,
)
MiniLM_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=MiniLM(task_name="LaMP_1"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "MiniLM.txt"),
    callback=task_1_parse_response,
)

In [4]:
def task_1_with_keywords_query(with_keyword_params, debug: bool = False):
    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    os.makedirs(store_dir, exist_ok=True)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120
    worker_count = 8 if not debug else 1
    api_keys = (
        [
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
        ]
        if not debug
        else None
    )

    ready_model = dict()

    for text_rank_top_k_keywords, bm25_top_k in with_keyword_params:
        file_ending = f"{entry_per_category}_{text_rank_top_k_keywords}_{bm25_top_k}"

        question_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_prompts_questions_with_keyword_{file_ending}.json",
        )
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_with_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                # "BERTSERINI": BERTSERINI_Subscriber,
                # "DistilBERT": DistilBERT_Subscriber,
                # "MiniLM": MiniLM_Subscriber,
                "OpenAI": OpenAI_Subscriber,
            },
            worker_count=worker_count,
            prompt_save_path=question_store_path,
            preds_save_path={
                # "BERTSERINI": os.path.join(store_dir,f"{task_header}_train_preds_BERTSERINI_with_keyword_{file_ending}.json"),
                # "DistilBERT": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_DistilBERT_with_keyword_{file_ending}.json",
                # ),
                # "MiniLM": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_MiniLM_with_keyword_{file_ending}.json",
                # ),
                "OpenAI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_OpenAI_with_keyword_{file_ending}.json",
                ),
            },
            keyword_extraction=True,
        )

        ready_model[(text_rank_top_k_keywords, bm25_top_k)] = curr_task
        curr_task.load_prompts(question_store_path)
        curr_task.subscribe(
            skip_eval=True,
            api_keys=api_keys,
        )


def task_1_with_keywords_eval(
    with_keyword_params, ready_models: Dict[Tuple[int, int], LaMPTask] = None
):
    if ready_models is not None:
        for (text_rank_top_k_keywords, bm25_top_k), curr_task in ready_models.items():
            curr_task.evaluate()
        return

    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120

    evaluated_result = dict()

    for text_rank_top_k_keywords, bm25_top_k in with_keyword_params:
        file_ending = f"{entry_per_category}_{text_rank_top_k_keywords}_{bm25_top_k}"
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_with_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                # "BERTSERINI": BERTSERINI_Subscriber,
                # "DistilBERT": DistilBERT_Subscriber,
                # "MiniLM": MiniLM_Subscriber,
                "OpenAI": OpenAI_Subscriber,
            },
        )
        curr_task.evaluate(
            preds_save_name={
                # "BERTSERINI": os.path.join(store_dir,f"{task_header}_train_preds_BERTSERINI_with_keyword_{file_ending}.json"),
                # "DistilBERT": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_DistilBERT_with_keyword_{file_ending}.json",
                # ),
                # "MiniLM": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_MiniLM_with_keyword_{file_ending}.json",
                # ),
                "OpenAI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_OpenAI_with_keyword_{file_ending}.json",
                ),
            }
        )
        print(curr_task.score)
        evaluated_result[f"{text_rank_top_k_keywords} {bm25_top_k}"] = curr_task.score
    return evaluated_result

In [5]:
with_keyword_params = [(5, 5), (10, 10)]
debug = False

In [6]:
# with_keyword_params = [(5, 5)]
with_keyword_params = [(10, 10)]
# debug=True

In [7]:
ready_models = task_1_with_keywords_query(with_keyword_params, debug=debug)

Output()

['0037', '00204', '00425', '00162', '009', '00832', '0042', '00394', '00692', '00336', '00421', '00930', '00936', 
'00884', '00765', '00854', '00980', '001239', '001015', '00392', '00663', '001321', '001294', '001051', '001472', 
'001470', '001309', '001285', '00888', '001048', '001283', '001812', '001954', '001508', '00549', '001700', 
'001519', '001751', '001778', '001659', '001335', '001878', '002071', '002238', '001028', '002175', '001972', 
'00781', '001600', '002215', '002004', '002270', '002564', '001985', '002673', '002393', '002064', '002724', 
'002302', '002247', '002169', '002686', '001329', '002973', '003007', '002791', '002190', '003391', '003052', 
'003126', '001544', '003147', '002927', '003405', '002923', '003660', '003705', '003858', '003500', '003718', 
'004116', '003779', '003137', '003459', '003864', '003699', '004178', '004119', '004557', '004017', '003978', 
'003220', '004575', '004366', '004276', '004740', '004569', '004539', '003211', '004639', '004993', '002678', 
'004846', '004707', '003363', '003897', '004568', '005235', '004757', '005287', '005093', '005307', '004187', 
'001975', '005298', '004791', '005051', '005232', '004853', '005096', '005182', '005569', '005527', '005419', 
'005705', '005273', '005380', '005647', '004913', '005940', '005323', '004903', '005799', '005656', '005747', 
'005425', '006011', '005375', '006204', '005758', '006284', '005849', '006371', '006520', '006044', '005904', 
'005661', '005751', '006564', '005320', '006588', '006053', '006428', '006735', '006521', '006261', '006456', 
'006932', '006258', '005195', '006776', '007195', '007097', '005775', '007026', '005301', '007212', '006864', 
'007220', '006998', '006646', '006631', '007503', '008106', '007283', '008182', '007058', '007263', '007160', 
'006972', '007461', '007890', '007543', '008279', '007200', '008303', '008161', '007356', '007563', '008527', 
'007745', '008325', '008418', '008437', '007986', '008577', '008511', '006772', '008649', '008488', '008393', 
'008690', '008833', '008439', '008324', '008453', '008986', '008964', '009019', '008921', '006973', '008299', 
'009101', '008809', '005787', '009393', '008646', '009097', '008758', '009263', '009147', '009517', '009670', 
'009639', '009434', '008757', '008800', '008742', '008834', '009232', '009471', '009293', '009446', '009659', 
'009140', '009595', '008884']

In [8]:
evaluate_results = task_1_with_keywords_eval(with_keyword_params)

{'OpenAI': {'accuracy': 0.5063291139240507, 'f1': 0.5218125960061444}}


In [21]:
def task_1_without_keywords_query(without_keyword_params, debug: bool = False):
    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    os.makedirs(store_dir, exist_ok=True)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120
    worker_count = 8 if not debug else 1
    api_keys = (
        [
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
        ]
        if not debug
        else None
    )

    ready_model = dict()

    for bm25_top_k in without_keyword_params:
        file_ending = f"{entry_per_category}_{bm25_top_k}"

        question_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_prompts_questions_without_keyword_{file_ending}.json",
        )
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_without_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                "BERTSERINI": BERTSERINI_Subscriber,
                # "DistilBERT": DistilBERT_Subscriber,
                # "MiniLM": MiniLM_Subscriber,
                # "OpenAI": OpenAI_Subscriber,
            },
            worker_count=worker_count,
            prompt_save_path=question_store_path,
            preds_save_path={
                "BERTSERINI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_BERTSERINI_without_keyword_{file_ending}.json",
                ),
                # "DistilBERT": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_DistilBERT_without_keyword_{file_ending}.json",
                # ),
                # "MiniLM": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_MiniLM_without_keyword_{file_ending}.json",
                # ),
                # "OpenAI": os.path.join(store_dir,f"{task_header}_train_preds_OpenAI_without_keyword_{file_ending}.json"),
            },
            keyword_extraction=False,
        )

        ready_model[bm25_top_k] = curr_task
        curr_task.load_prompts(question_store_path)
        curr_task.subscribe(
            skip_eval=True,
            api_keys=api_keys,
        )


def task_1_without_keywords_eval(
    without_keyword_params, ready_models: Dict[Tuple[int, int], LaMPTask] = None
):
    if ready_models is not None:
        for bm25_top_k, curr_task in ready_models.items():
            curr_task.evaluate()
        return

    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120

    evaluated_result = dict()

    for bm25_top_k in without_keyword_params:
        file_ending = f"{entry_per_category}_{bm25_top_k}"
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_without_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                "BERTSERINI": BERTSERINI_Subscriber,
                # "DistilBERT": DistilBERT_Subscriber,
                # "MiniLM": MiniLM_Subscriber,
                # "OpenAI": OpenAI_Subscriber,
            },
        )
        curr_task.evaluate(
            preds_save_name={
                "BERTSERINI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_BERTSERINI_without_keyword_{file_ending}.json",
                ),
                # "DistilBERT": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_DistilBERT_without_keyword_{file_ending}.json",
                # ),
                # "MiniLM": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_MiniLM_without_keyword_{file_ending}.json",
                # ),
                # "OpenAI": os.path.join(store_dir,f"{task_header}_train_preds_OpenAI_without_keyword_{file_ending}.json"),
            }
        )
        print(curr_task.score)
        evaluated_result[f"{bm25_top_k}"] = curr_task.score
    return evaluated_result

In [28]:
debug = False
without_keyword_params = [2, 4]
without_keyword_params = [4]

In [30]:
ready_models_no_keywords = task_1_without_keywords_query(
    without_keyword_params, debug=debug
)

Output()

['00188', '00264', '00250', '009', '00387', '0060', '00227', '00371', '00467', '00521', '00531', '00544', '00847', 
'00605', '00594', '00625', '00591', '00916', '00914', '00961', '00983', '001111', '001231', '00982', '001146', 
'001238', '00973', '001275', '001350', '001335', '001326', '001346', '001413', '001428', '001404', '001479', 
'001606', '001662', '001677', '001839', '001831', '001589', '001722', '001634', '002024', '002136', '002224', 
'002229', '001929', '002422', '001985', '002017', '002532', '002662', '002785', '002672', '002681', '002614', 
'002568', '002664', '002819', '002993', '002814', '003127', '002842', '002950', '003003', '002797', '003253', 
'003186', '003247', '003137', '003232', '003275', '003311', '003203', '003384', '003332', '003549', '003495', 
'003614', '003316', '003413', '003398', '003971', '003814', '003924', '003912', '003833', '003636', '003733', 
'003941', '003999', '004158', '004173', '004065', '004305', '004097', '004166', '004195', '004680', '004402', 
'004418', '004406', '004414', '004724', '004553', '004337', '004765', '004730', '004886', '004885', '004832', 
'004786', '004739', '004840', '005035', '005204', '004920', '005021', '005073', '005208', '004918', '005219', 
'005516', '005367', '005489', '005353', '005631', '005436', '005777', '005764', '005930', '006090', '005970', 
'006079', '005879', '005835', '005952', '005959', '006297', '006286', '006215', '006194', '006298', '006129', 
'006311', '006332', '006808', '006445', '006632', '006339', '006915', '006476', '006373', '006493', '006991', 
'007048', '007023', '007054', '007037', '007051', '006977', '007069', '007404', '007326', '007296', '007202', 
'007315', '007184', '007250', '007225', '007405', '007853', '007420', '007894', '007749', '007618', '007741', 
'007623', '007718', '007939', '007935', '007973', '008217', '008222', '008319', '007963', '008017', '008019', 
'008238', '008261', '008257', '008341', '008349', '008364', '008493', '008519', '008585', '008347', '008426', 
'008484', '008360', '008525', '008481', '008594', '008608', '008796', '008824', '008617', '008735', '008722', 
'008675', '008676', '008804', '008828', '008985', '008978', '008956', '009001', '009060', '008975', '009095', 
'009241', '009123', '009163', '009117', '009131', '009235', '009314', '009659', '009470', '009380', '009680']

In [31]:
evaluate_results_no_keywords = task_1_without_keywords_eval(without_keyword_params)

{'BERTSERINI': {'accuracy': 0.3191489361702128, 'f1': 0.2987044882421563}}
